# Lesson 02: coordinate systems

*This lesson made by Simon M Mudd and last updated 21/11/2021*

In lesson 01 we learned how to download data, look at the information about the data (e.g., its dimensions, its *CRS*, the size of the pixels) and make simple plots. 

In this lesson we will look at coordinate reference systems. 

## First import some stuff we need

In [2]:
import rasterio as rio
import matplotlib.pyplot as plt

## Lets look at the data from the last lession

In the last lesson we grabbed a raster from [opentopography](https://opentopography.org/). 
If you haven't done that lesson, go back and get the raster. It is called `rio_aguas_SRTMGL1.tif`

We can see if it is in this directory by calling the `!ls` command

In [1]:
!ls

Lesson_01_getting_topographic_data.ipynb
Lesson_02_coordinate_systems.ipynb
Lesson_03_simple_metrics_and_resolution.ipynb
Lesson_04_basic_topographic_features.ipynb
rio_aguas_SRTMGL1.tif
rio_aguas_SRTMGL1_HS.tif


Now lets go back to the pixel sizes of the raster. I'll get them with rasterio:

In [4]:
filename = "rio_aguas_SRTMGL1.tif"
with rio.open(filename) as src:
    print("\nRaster pixel sizes in the X,Y directions are:")
    print(src.res)


Raster pixel sizes in the X,Y directions are:
(0.0002777777777778146, 0.0002777777777778146)


What are these? 

SRTM data is available in GL1 and GL3 versions from OpenTopography. The "GL" stands for global and the 1 and 3 stand for 1 and 3 arcseconds, respectively. The arcsecond is the size of the pixels. 

[Read about arcseconds here](https://en.wikipedia.org/wiki/Minute_and_second_of_arc)

If you don't want to read that, basically an arcsecond is 1/3600 of a degree, which is 0.000277 in decimal degree format.

So the pixel size in this DEM is in decimal degrees. 

1 arcsecond in decimal degrees is around 30 m at the equator. It changes as you change latitude. At high latitudes the X and Y dimensions will always be 1 arcsecond but the length in metres will change. 

This is a bit of a problem for us because we like to do things like measure gradients and areas and distances in units like metres and metres squared: measuring, say, a river gradient in metres per arcsecond would seem rather silly. 

This brings us to projections, transformations, and coordinate reference systems.

## Projections, transformations, and coordinate reference systems

I am not going to get into too much detail here, but the Earth is a sphere so one way to figure out where you are on the sphere is to use geographic coordinates: latitude and longitude. 

But you can also project this spherical surface onto a flat surface (e.g., a paper map). This is called a projection. A projection can allow you to convert your coordinates into distances. 

[You can spend all day reading about projections](https://en.wikipedia.org/wiki/Map_projection)

